In [1]:
import glob
import re
import matplotlib as plt
import numpy as np
import scipy as sp
import xarray as xr
import intake
import intake_esm
import pandas as pd

In [4]:
from dask_jobqueue import PBSCluster
from distributed import Client

In [10]:
######## File paths ################
lustre_scratch  = "/lustre/desc1/scratch/harshah"
gdex_data       = "/gdex/data/"
# gdex_url      = 'https://data.gdex.ucar.edu/'
eol_data        = "/gdex/data/special_projects/pythia_2025/eol-cookbook/m2hats_iss2_data/prof449Mhz_30min_winds/"
#########
era5_catalog      =  gdex_data + 'special_projects/pythia_2024/pythia_intake_catalogs/era5_catalog.json'
print(era5_catalog)

/gdex/data/special_projects/pythia_2024/pythia_intake_catalogs/era5_catalog.json


In [7]:
cluster = PBSCluster(
        job_name = 'dask-eol-25',
        cores = 1,
        memory = '4GiB',
        processes = 1,
        local_directory = lustre_scratch + '/dask/spill',
        log_directory = lustre_scratch + '/dask/logs/',
        resource_spec = 'select=1:ncpus=1:mem=4GB',
        queue = 'casper',
        walltime = '3:00:00',
        #interface = 'ib0'
        interface = 'ext')

In [8]:
client = Client(cluster)

In [9]:
# Scale the cluster and display cluster dashboard URL
n_workers =5
cluster.scale(n_workers)
client.wait_for_workers(n_workers = n_workers)
cluster

PBSCluster(a0d29453, 'tcp://128.117.208.96:45699', workers=5, threads=5, memory=20.00 GiB)

### Load data

In [25]:
%%time
prof449_wind = xr.open_mfdataset(eol_data + '*.nc',concat_dim = 'time',combine='nested')

ValueError: cannot reindex or align along dimension 'height' because of conflicting dimension sizes: {97, 61, 46, 55}

In [29]:
prof449Mhz_wind_test = xr.open_dataset(eol_data + 'prof449.20230927.winds.30.nc')
prof449Mhz_wind_test

<xarray.Dataset> Size: 102kB
Dimensions:       (time: 48, height: 46)
Coordinates:
  * time          (time) datetime64[ns] 384B 2023-09-27T00:15:00 ... 2023-09-...
    height        (time, height) float32 9kB ...
Data variables: (12/18)
    base_time     datetime64[ns] 8B ...
    lat           float32 4B ...
    lon           float32 4B ...
    alt           float32 4B ...
    frequency     float32 4B ...
    x_spacing     float32 4B ...
    ...            ...
    snrw          (time, height) float32 9kB ...
    specWid       (time, height) float32 9kB ...
    u_dispersion  (time, height) float32 9kB ...
    v_dispersion  (time, height) float32 9kB ...
    w_dispersion  (time, height) float32 9kB ...
    cons_npoints  (time, height) int16 4kB ...
Attributes: (12/16)
    description:        Modular Profiler consensus winds
    Processing-Method:  30 minute consensus averages
    Processing-Ops:       read_raw_winds mapr_nc szl yyyy correct_szl correct...
    history:            write_cons_5.pro
    author:             Bill Brown, NCAR/EOL
    date:               Mon Jul  1 10:04:38 2024   1719849878
    ...                 ...
    project:            M2HATS
    site:               Tonopah Airport, NV
    missing_value:      -999.0
    zebra_platform:     MAPR/449 winds
    first_file:         /scr/isf_apg/raw/m2hats/iss1/modular_prof/reprocessed...
    last_file:          /scr/isf_apg/raw/m2hats/iss1/modular_prof/reprocessed...

### Load ERA5 data

In [18]:
%%time
era5_cat = intake.open_esm_datastore(era5_catalog)
era5_cat

/glade/u/home/harshah/venvs/osdf/lib/python3.10/site-packages/intake_esm/cat.py:250: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


CPU times: user 1.86 s, sys: 235 ms, total: 2.1 s
Wall time: 2.22 s


,unique
era_id,1
datatype,2
level_type,1
step_type,7
table_code,4
param_code,164
variable,212
long_name,212
units,33
year,85


In [23]:
era5_cat.df['variable'].unique()

array(['PV', 'CRWC', 'CSWC', 'Z', 'T', 'U', 'V', 'Q', 'W', 'VO', 'D', 'R',
       'O3', 'CLWC', 'CIWC', 'CC', 'ALUVP', 'ALUVD', 'ALNIP', 'ALNID',
       'CI', 'ASN', 'RSN', 'SSTK', 'ISTL1', 'ISTL2', 'ISTL3', 'ISTL4',
       'SWVL1', 'SWVL2', 'SWVL3', 'SWVL4', 'CAPE', 'LAILV', 'LAIHV',
       'TCLW', 'TCIW', 'SP', 'TCW', 'TCWV', 'STL1', 'SD', 'CHNK', 'MSL',
       'BLH', 'TCC', 'VAR_10U', 'VAR_10V', 'VAR_2T', 'VAR_2D', 'STL2',
       'STL3', 'LCC', 'MCC', 'HCC', 'SRC', 'TCO3', 'IEWS', 'INSS', 'ISHF',
       'IE', 'SKT', 'STL4', 'TSN', 'FAL', 'FSR', 'FLSR', 'LBLT', 'LTLT',
       'LSHF', 'LICT', 'LICD', 'TCRW', 'TCSW', 'U10N', 'V10N', 'VAR_100U',
       'VAR_100V', 'LMLT', 'LMLD', 'VIMA', 'VIT', 'VIKE', 'VITHE',
       'VIPIE', 'VIPILE', 'VITOE', 'VIEC', 'VIMAE', 'VIMAN', 'VIKEE',
       'VIKEN', 'VITHEE', 'VITHEN', 'VIWVE', 'VIWVN', 'VIGE', 'VIGN',
       'VITOEE', 'VITOEN', 'VIOZE', 'VIOZN', 'VILWD', 'VIIWD', 'VIMAD',
       'VIKED', 'VITHED', 'VIWVD', 'VIGD', 'VITOED', 'VIOZD', 'VILWE

In [ ]:
temp_cat = era5_cat.search(variable='VAR_2T',frequency = 'hourly',year=2000)
temp_cat